### Preprocesamiento

In [177]:
import pandas as pd
from opencage.geocoder import OpenCageGeocode
import json


conversión de valor de texto de los test a enteros, asignar 'Zalamea De La Serena' al codigo postal 6430 y descartar filas sin codigo postal ni geo location

In [178]:
# Definir el diccionario de mapeo
value_test_mapping = {
    'Negativo': 0,
    'Positivo': 1,
    'Positivo Fuerte': 1
}

# Leer el archivo CSV
file_path = "data/new_data.csv" 
data = pd.read_csv(file_path)

In [179]:

data["Date"] = data["Date"].astype(str).str.replace(r'[^\w\s]', '', regex=True)  # Eliminar caracteres especiales
data["Date"] = pd.to_datetime(data["Date"], errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')

data['Value test'] = data['Value test'].map(value_test_mapping)
data['Value test'] = data['Value test'].fillna(0).astype(int)

In [180]:
# Eliminar filas donde 'Postal code' y 'Geo Location' estén vacíos
data_cleaned = data.dropna(subset=['Postal code'], how='all')



C:\Users\pablo\AppData\Local\Temp\ipykernel_23828\289051000.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['Postal code'] = data_cleaned['Postal code'].astype(str).str.split('.').str[0]


In [181]:
# Mapear y convertir valores de 'Value test'
val_no_deseados = ['Dudoso', 'Error', 'Invalido', 'Inválido']
df_cleaned = data[~data['Value test'].isin(val_no_deseados)]

In [182]:
data_cleaned['Postal code'] = data_cleaned['Postal code']

data_cleaned.loc[data_cleaned['Postal code'] == '6430', 'City'] = 'Zalamea De La Serena'

C:\Users\pablo\AppData\Local\Temp\ipykernel_23828\1219181853.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['Postal code'] = data_cleaned['Postal code']


Hace consultas a la api de "open cage data" (https://opencagedata.com/) para aquellas columnas que tienen geo location peor no tienen codigo postal, obtienen este último a través de la localización.

In [183]:
# Inicializa el geocoder con la clave de API
#OCG = OpenCageGeocode('8f636a0d28d14db099c8af16ba33ebac')

In [184]:
# Función para obtener el código postal a partir de Geo Location
"""
def get_postal_code(geo_location):
    try:
        # Convertir la cadena JSON en un diccionario
        location = json.loads(geo_location)
        lat = location['latitude']
        lng = location['longitude']
        # Realizar la búsqueda inversa
        results = OCG.reverse_geocode(lat, lng)
        if results and 'postcode' in results[0]['components']:
            return results[0]['components']['postcode']
    except Exception as e:
        print(f"Error procesando {geo_location}: {e}")
    return None"""

'\ndef get_postal_code(geo_location):\n    try:\n        # Convertir la cadena JSON en un diccionario\n        location = json.loads(geo_location)\n        lat = location[\'latitude\']\n        lng = location[\'longitude\']\n        # Realizar la búsqueda inversa\n        results = OCG.reverse_geocode(lat, lng)\n        if results and \'postcode\' in results[0][\'components\']:\n            return results[0][\'components\'][\'postcode\']\n    except Exception as e:\n        print(f"Error procesando {geo_location}: {e}")\n    return None'

Hace consultas a la api de "open cage data" (https://opencagedata.com/) para aquellas columnas que tienen geo location peor no tienen geolocalizacion y la obtiene a traves de su codigo postal.

Agrupa en el csv archivo agrupado todos los codigos postales y suma el valor de los resultados de los tests.

In [185]:
"""
def get_geo_location(postal_code):
    try:
        # Convertir el código postal a cadena
        postal_code_str = str(int(postal_code))  # Eliminar decimales innecesarios
        results = OCG.geocode(postal_code_str)
        if results:
            lat = results[0]['geometry']['lat']
            lng = results[0]['geometry']['lng']
            return f"{lat},{lng}"
    except Exception as e:
        print(f"Error procesando {postal_code}: {e}")
    return None"""

'\ndef get_geo_location(postal_code):\n    try:\n        # Convertir el código postal a cadena\n        postal_code_str = str(int(postal_code))  # Eliminar decimales innecesarios\n        results = OCG.geocode(postal_code_str)\n        if results:\n            lat = results[0][\'geometry\'][\'lat\']\n            lng = results[0][\'geometry\'][\'lng\']\n            return f"{lat},{lng}"\n    except Exception as e:\n        print(f"Error procesando {postal_code}: {e}")\n    return None'

In [186]:
#Agrupar por Código Postal
df_grouped = df_cleaned.groupby("Postal code").agg(
    total_tests=('Value test', 'count'),
    total_positives=('Value test', lambda x: (x == 1).sum())
).reset_index()

In [187]:
#Guardar los Resultados
df_cleaned.to_csv("data/preprocessed_data.csv", index=False)  # CSV limpio
df_grouped.to_csv("data/code_test_results.csv", index=False)  # Agrupado por código postal
